<a href="https://colab.research.google.com/github/serinamarie/DS-Unit-4-Sprint-3-Deep-Learning/blob/master/LS_DS_Uni_4_Sprint_3_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Major Neural Network Architectures Challenge
## *Data Science Unit 4 Sprint 3 Challenge*

In this sprint challenge, you'll explore some of the cutting edge of Data Science. This week we studied several famous neural network architectures: 
recurrent neural networks (RNNs), long short-term memory (LSTMs), convolutional neural networks (CNNs), and Autoencoders. In this sprint challenge, you will revisit these models. Remember, we are testing your knowledge of these architectures not your ability to fit a model with high accuracy. 

__*Caution:*__  these approaches can be pretty heavy computationally. All problems were designed so that you should be able to achieve results within at most 5-10 minutes of runtime on SageMaker, Colab or a comparable environment. If something is running longer, doublecheck your approach!

## Challenge Objectives
*You should be able to:*
* <a href="#p1">Part 1</a>: Train a LSTM classification model
* <a href="#p2">Part 2</a>: Utilize a pre-trained CNN for objective detection
* <a href="#p3">Part 3</a>: Describe the components of an autoencoder
* <a href="#p4">Part 4</a>: Describe yourself as a Data Science and elucidate your vision of AI

<a id="p1"></a>
## Part 1 - RNNs

Use an RNN/LSTM to fit a multi-class classification model on reuters news articles to distinguish topics of articles. The data is already encoded properly for use in an RNN model. 

Your Tasks: 
- Use Keras to fit a predictive model, classifying news articles into topics. 
- Report your overall score and accuracy

For reference, the [Keras IMDB sentiment classification example](https://github.com/keras-team/keras/blob/master/examples/imdb_lstm.py) will be useful, as well the RNN code we used in class.

__*Note:*__  Focus on getting a running model, not on maxing accuracy with extreme data size or epoch numbers. Only revisit and push accuracy if you get everything else done!

In [0]:
from tensorflow.keras.datasets import reuters

(X_train, y_train), (X_test, y_test) = reuters.load_data(num_words=None,
                                                         skip_top=0,
                                                         maxlen=None,
                                                         test_split=0.2,
                                                         seed=723812,
                                                         start_char=1,
                                                         oov_char=2,
                                                         index_from=3)

In [231]:
# Demo of encoding

word_index = reuters.get_word_index(path="reuters_word_index.json")

print(f"Iran is encoded as {word_index['iran']} in the data")
print(f"London is encoded as {word_index['london']} in the data")
print("Words are encoded as numbers in our dataset.")

Iran is encoded as 779 in the data
London is encoded as 544 in the data
Words are encoded as numbers in our dataset.


In [232]:
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM

batch_size = 46
max_features = len(word_index.values())
maxlen = 200

print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

8982 train sequences
2246 test sequences
Pad sequences (samples x time)
X_train shape: (8982, 200)
X_test shape: (2246, 200)


In [238]:
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(46, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=3,
          validation_data=(X_test, y_test))

score, acc = model.evaluate(X_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Train...
Train on 8982 samples, validate on 2246 samples
Epoch 1/3
8982/8982 [==============================] - 91s 10ms/sample - loss: 2.1657 - acc: 0.4453 - val_loss: 2.2682 - val_acc: 0.5036
Epoch 2/3
8982/8982 [==============================] - 74s 8ms/sample - loss: 1.7490 - acc: 0.5474 - val_loss: 1.6027 - val_acc: 0.5886
Epoch 3/3
2246/2246 [==============================] - 4s 2ms/sample - loss: 1.5150 - acc: 0.6118
Test score: 1.514996928236682
Test accuracy: 0.61175424


## Sequence Data Question
#### *Describe the `pad_sequences` method used on the training dataset. What does it do? Why do you need it?*

Keras needs sequences of the same length for computation. Padding the sequences with zeroes ensures that they are all of the same length, as the model will learn over time that the zeroes carry no information.

## RNNs versus LSTMs
#### *What are the primary motivations behind using Long-ShortTerm Memory Cell unit over traditional Recurrent Neural Networks?*

Because of the vanishing and exploding gradient problem, an LSTM is preferable when it is important to maintain information over a longer period of time. LSTMs have a cell state which has gates that regulate the ability of information to leave and enter the cell. Therefore, LSTMs are better at handling long-term dependency.

## RNN / LSTM Use Cases
#### *Name and Describe 3 Use Cases of LSTMs or RNNs and why they are suited to that use case*

RNNs are good at predicting sequences. Thus, one use case would be using text data to generate/predict new text. Another use case is on speech data, perhaps for speech recognition, since RNNs are good at processing sequenced data.

<a id="p2"></a>
## Part 2- CNNs

### Find the Frog

Time to play "find the frog!" Use Keras and ResNet50 (pre-trained) to detect which of the following images contain frogs:

<img align="left" src="https://d3i6fh83elv35t.cloudfront.net/newshour/app/uploads/2017/03/GettyImages-654745934-1024x687.jpg" width=400>


In [0]:
!pip install google_images_download

In [204]:
from google_images_download import google_images_download

response = google_images_download.googleimagesdownload()
arguments = {"keywords": 'lily frog pond', "limit": 60, "print_urls": True}
a = 'frog pond lily'

# def downloadimages(query):
#     for x in range(35,80):
#         arguments = {"keywords": query, "limit":x, 'print_urls': True} 
#         try:
#             response.download(arguments)
#         except:
#             pass

absolute_image_paths = response.download(arguments)


Item no.: 1 --> Item name = lily frog pond
Evaluating...
Starting Download...
Image URL: https://www.maxpixel.net/static/photo/1x/Nature-Leaf-Animal-Water-Lily-Frog-Pond-4297486.jpg
URLError on an image...trying next one... Error: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:852)>
Image URL: https://www.seekpng.com/png/detail/78-780997_pond-transparent-lily-pad-clipart-frog-pond-clip.png
Completed Image ====> 1.78-780997_pond-transparent-lily-pad-clipart-frog-pond-clip.png
Image URL: https://image.shutterstock.com/image-photo/green-frogs-pond-lilly-pads-260nw-50197960.jpg
Completed Image ====> 2.green-frogs-pond-lilly-pads-260nw-50197960.jpg
Image URL: http://www.slrobertson.com/images/usa/georgia/atlanta/atl-botanical-gardens/frog-lily-pond-3-b.jpg
Completed Image ====> 3.frog-lily-pond-3-b.jpg
Image URL: https://i.redd.it/awd8c9t1abg11.jpg
Completed Image ====> 4.awd8c9t1abg11.jpg
Image URL: https://cf.geekdo-images.com/opengraph_letterbox/img/CP

At time of writing at least a few do, but since the Internet changes - it is possible your 5 won't. You can easily verify yourself, and (once you have working code) increase the number of images you pull to be more sure of getting a frog. Your goal is to validly run ResNet50 on the input images - don't worry about tuning or improving the model.

*Hint* - ResNet 50 doesn't just return "frog". The three labels it has for frogs are: `bullfrog, tree frog, tailed frog`

*Stretch goals* 
- Check for fish or other labels
- Create a matplotlib visualizations of the images and your prediction as the visualization label

In [0]:
import numpy as np

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

def process_img_path(img_path):
  return image.load_img(img_path, target_size=(224, 224))

def imread_convert(f):
    return imread(f).astype(np.uint8)

def img_contains_frog(img):
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    model = ResNet50(weights='imagenet')
    features = model.predict(x)
    results = decode_predictions(features, top=3)[0]
    print(results)
    if (results[0][1] == 'bullfrog') or (results[1][1] == 'tree_frog') or (results[1][1] == 'tailed_frog'):
        print(True)
    else:
        print(False)

In [245]:
for q in absolute_image_paths[0]['lily frog pond'][5:15]:
    img_contains_frog(process_img_path(q))

[('n01641577', 'bullfrog', 0.98556846), ('n01644900', 'tailed_frog', 0.01382296), ('n01644373', 'tree_frog', 0.00058391516)]
True
[('n03291819', 'envelope', 0.34432983), ('n04116512', 'rubber_eraser', 0.10736573), ('n03876231', 'paintbrush', 0.06958141)]
False
[('n02834397', 'bib', 0.5220779), ('n02730930', 'apron', 0.12062734), ('n03485794', 'handkerchief', 0.11799439)]
False
[('n06359193', 'web_site', 0.3861849), ('n03291819', 'envelope', 0.3517924), ('n03642806', 'laptop', 0.13926636)]
False
[('n07718747', 'artichoke', 0.11814717), ('n02009229', 'little_blue_heron', 0.11751351), ('n02011460', 'bittern', 0.047491435)]
False
[('n01737021', 'water_snake', 0.16246946), ('n03991062', 'pot', 0.13198888), ('n07742313', 'Granny_Smith', 0.06185926)]
False
[('n01737021', 'water_snake', 0.115632735), ('n03457902', 'greenhouse', 0.08579437), ('n01667778', 'terrapin', 0.07893841)]
False
[('n01641577', 'bullfrog', 0.25773758), ('n02229544', 'cricket', 0.11621964), ('n02268443', 'dragonfly', 0.114

<a id="p3"></a>
## Part 3 - Autoencoders

Describe a use case for an autoencoder given that an autoencoder tries to predict its own input. 

Denoising images is a use case for an autoencoder. It can also detect outliers by checking for anomalies. 

<a id="p4"></a>
## Part 4 - More...

Answer the following questions, with a target audience of a fellow Data Scientist:

- What do you consider your strongest area, as a Data Scientist?
I'm honestly not sure. Perhaps presenting data in an appealing way.
- What area of Data Science would you most like to learn more about, and why?
I'd be interested in understanding NLP and NN better.

- Where do you think Data Science will be in 5 years?
I have no idea but hopefully in a good place.
- What are the threats posed by AI to our society?
We could suffer from a lack of privacy, too much surveillance, biases.
- How do you think we can counteract those threats? 
Thinking our actions through the long-term, accountability.
- Do you think achieving General Artifical Intelligence is ever possible?
In some ways, I think it is possible now. But can those tasks be performed as quickly as humans, I'm unsure.

A few sentences per answer is fine - only elaborate if time allows.

## Congratulations! 

Thank you for your hard work, and congratulations! You've learned a lot, and you should proudly call yourself a Data Scientist.
